In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
from pandas import read_csv
import numpy as np

In [75]:
import plotly.express as px
from sklearn.preprocessing import OneHotEncoder

In [76]:
# import dataset
filename = "bondugula2-6.csv"
raw_data = read_csv(filename, header=1)
print(raw_data.columns)

Index(['Protein', 'No.', 'Res', 'isUnstruct', 'E6', 'E20', 'E22', 'Vkbat',
       'chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred',
       '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U'],
      dtype='object')


# Data Preprocessing

In [77]:
# clean dataset
data = raw_data.drop(['E6', 'E20'], axis=1)
data = data[data['Res']!='_']

In [78]:

data['num_X'] = data['HAS_H'] + data['HAS_S'] + data['HAS_O'] + data['HAS_U'] # column with num
data['is_switch'] = np.where(data['num_X'] > 1, 1, 0) # create switch where contains multiple X


In [79]:
data.tail(20)

,Protein,No.,Res,isUnstruct,E22,Vkbat,chou_fasman,sspro_5,gor4,dsc,jnet,psipred,# homologues,HAS_H,HAS_S,HAS_O,HAS_U,num_X,is_switch
5040,1CY5,78,E,0.064506,NaN,4.0,Other,Helix,Helix,Helix,Other,Other,20,0,0,1,0,1,0
5041,1CY5,79,G,0.066382,NaN,3.0,Other,Other,Helix,Helix,Other,Other,20,0,0,1,0,1,0
5042,1CY5,80,Y,0.064804,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,0,0,0,1,1,0
5043,1CY5,81,K,0.079351,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5044,1CY5,82,D,0.085181,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5045,1CY5,83,L,0.083146,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5046,1CY5,84,A,0.091622,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5047,1CY5,85,A,0.099122,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5048,1CY5,86,L,0.105681,NaN,1.0,Helix,Helix,Helix,Helix,Helix,Helix,20,1,0,0,0,1,0
5049,1CY5,87,L,0.126871,NaN,3.0,Other,Helix,Helix,Other,Helix,Helix,20,1,0,0,0,1,0


# Switches Initial Visualization

In [80]:
# Plotting with plotly

# dot plot
fig = px.scatter(data, x="Protein", y="No.", color=data["is_switch"].astype(str), hover_data=['Res'])
fig.update_traces(marker={"opacity": 1.0})
fig.update_xaxes(type='category')
fig.show()


In [81]:
print(data.describe())
print(data.dtypes)

               No.   isUnstruct  E22        Vkbat  # homologues        HAS_H  \
count  5035.000000  5035.000000  0.0  5035.000000   5035.000000  5035.000000   
mean    129.993843     0.265215  NaN     2.719325     19.250645     0.475670   
std     102.963424     0.253149  NaN     1.613570     23.140653     0.499457   
min       1.000000     0.001916  NaN     1.000000      1.000000     0.000000   
25%      51.000000     0.068755  NaN     1.000000      7.000000     0.000000   
50%     105.000000     0.175291  NaN     2.400000     11.000000     0.000000   
75%     185.000000     0.378653  NaN     3.000000     26.000000     1.000000   
max     516.000000     1.000000  NaN     9.000000    119.000000     1.000000   

             HAS_S        HAS_O        HAS_U        num_X    is_switch  
count  5035.000000  5035.000000  5035.000000  5035.000000  5035.000000  
mean      0.282026     0.335452     0.333863     1.427011     0.321946  
std       0.450031     0.472195     0.471638     0.687872   

In [82]:
percent_switches = (data['is_switch'].sum())/(len(data['is_switch']))
print("Percentage of proteins in dataset that are switches is " + str(round(percent_switches*100,2)) + "%")

Percentage of proteins in dataset that are switches is 32.19%


# Data Standardization & Transformation

In [83]:
def normalize_column(quant_column):
    magnitude = np.sqrt((np.power(quant_column,2)).sum())
    return quant_column/magnitude

def standardize_column(quant_column):
    U = np.mean(quant_column)
    o = np.std(quant_column)
    return (quant_column - U)/o

In [84]:
# Standardize quantitative data

standardized_data = data
standardized_data['isUnstruct'] = standardize_column(data['isUnstruct'])
standardized_data['Vkbat'] = standardize_column(data['Vkbat'])
standardized_data['# homologues'] = standardize_column(data['# homologues'])

In [85]:
# Convert residues to standardized numeric
standardized_data['Res_numeric'] = standardized_data['Res'].apply(lambda x: ord(str(x)))
standardized_data['Res_numeric'] = standardize_column(standardized_data['Res_numeric'])

In [86]:
standardized_data.describe()

,No.,isUnstruct,E22,Vkbat,# homologues,HAS_H,HAS_S,HAS_O,HAS_U,num_X,is_switch,Res_numeric
count,5035.000000,5.035000e+03,0.0,5.035000e+03,5.035000e+03,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5035.000000,5.035000e+03
mean,129.993843,1.016069e-16,NaN,2.201483e-16,4.515862e-17,0.475670,0.282026,0.335452,0.333863,1.427011,0.321946,-3.655026e-16
std,102.963424,1.000099e+00,NaN,1.000099e+00,1.000099e+00,0.499457,0.450031,0.472195,0.471638,0.687872,0.467269,1.000099e+00
min,1.000000,-1.040199e+00,NaN,-1.065646e+00,-7.887616e-01,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-1.627312e+00
25%,51.000000,-7.761409e-01,NaN,-1.065646e+00,-5.294519e-01,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,-8.864876e-01
50%,105.000000,-3.552581e-01,NaN,-1.979191e-01,-3.565787e-01,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,2.501293e-03
75%,185.000000,4.481514e-01,NaN,1.739640e-01,2.916955e-01,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,8.914902e-01
max,516.000000,2.902871e+00,NaN,3.892795e+00,4.310996e+00,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.928644e+00


# One Hot Encoding

In [87]:
standardized_data['chou_fasman'].unique()

array(['Other', 'Helix', 'Sheet'], dtype=object)

In [88]:
ohe = OneHotEncoder()
ohe_features_array = ohe.fit_transform(standardized_data[['chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred']]).toarray()
print(ohe_features_array)
print(len(ohe_features_array))
feature_labels = ohe.categories_
print(feature_labels)

[[0. 1. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 1. 0.]
 ...
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 1. ... 0. 1. 0.]
 [0. 1. 0. ... 0. 1. 0.]]
5035
[array(['Helix', 'Other', 'Sheet'], dtype=object), array(['Helix', 'Other', 'Sheet'], dtype=object), array(['Helix', 'Other', 'Sheet'], dtype=object), array(['Helix', 'Other', 'Sheet'], dtype=object), array(['Helix', 'Other', 'Sheet'], dtype=object), array(['Helix', 'Other', 'Sheet'], dtype=object)]


In [89]:
feature_labels = ohe.get_feature_names_out(['chou_fasman', 'sspro_5', 'gor4', 'dsc', 'jnet', 'psipred'])
print(feature_labels)

['chou_fasman_Helix' 'chou_fasman_Other' 'chou_fasman_Sheet'
 'sspro_5_Helix' 'sspro_5_Other' 'sspro_5_Sheet' 'gor4_Helix' 'gor4_Other'
 'gor4_Sheet' 'dsc_Helix' 'dsc_Other' 'dsc_Sheet' 'jnet_Helix'
 'jnet_Other' 'jnet_Sheet' 'psipred_Helix' 'psipred_Other' 'psipred_Sheet']


In [90]:
# test inverse transformation
ohe.inverse_transform([[0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0]])

array([['Other', 'Helix', 'Helix', 'Helix', 'Sheet', 'Other']],
      dtype=object)

In [91]:
ohe_df = pd.DataFrame(ohe_features_array, columns = feature_labels)
print(ohe_df)

      chou_fasman_Helix  chou_fasman_Other  chou_fasman_Sheet  sspro_5_Helix  \
0                   0.0                1.0                0.0            0.0   
1                   0.0                1.0                0.0            0.0   
2                   0.0                1.0                0.0            0.0   
3                   0.0                1.0                0.0            0.0   
4                   0.0                1.0                0.0            0.0   
...                 ...                ...                ...            ...   
5030                0.0                0.0                1.0            0.0   
5031                0.0                0.0                1.0            0.0   
5032                0.0                0.0                1.0            0.0   
5033                0.0                0.0                1.0            0.0   
5034                0.0                1.0                0.0            0.0   

      sspro_5_Other  sspro_5_Sheet  gor

In [97]:
standardized_data = standardized_data.reset_index() # for clean merging of the data frames

In [121]:
df_mo = pd.concat(
    [standardized_data, ohe_df],
    axis=1)
df_mo = df_mo.drop(columns=['index'])
df_mo

,Protein,No.,Res,isUnstruct,E22,Vkbat,chou_fasman,sspro_5,gor4,dsc,...,gor4_Sheet,dsc_Helix,dsc_Other,dsc_Sheet,jnet_Helix,jnet_Other,jnet_Sheet,psipred_Helix,psipred_Other,psipred_Sheet
0,1N62,1,M,2.767919,NaN,-1.065646,Other,Other,Other,Other,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,1N62,2,I,2.489836,NaN,-1.065646,Other,Other,Other,Other,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,1N62,3,P,2.402347,NaN,-1.065646,Other,Other,Other,Other,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,1N62,4,G,2.233079,NaN,-1.065646,Other,Other,Other,Other,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,1N62,5,S,2.044834,NaN,-1.065646,Other,Other,Other,Other,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5030,1CY5,93,V,0.447105,NaN,-1.065646,Sheet,Sheet,Sheet,Sheet,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
5031,1CY5,94,V,0.781769,NaN,-1.065646,Sheet,Sheet,Sheet,Sheet,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
5032,1CY5,95,S,1.492115,NaN,0.173964,Sheet,Other,Sheet,Sheet,...,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
5033,1CY5,96,V,1.876323,NaN,0.173964,Sheet,Other,Sheet,Other,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [122]:
# checking whether to ohe protein names
df_mo['Protein'].unique()

array(['1N62', '2A2L', '1SAC', '2A6Q', '2A6Z', '2A7K', '1U20', '2ABW',
       '2ZBL', '2AHO', '2AHR', '2ANE', '3SDH', '2AWI', '2AXP', '2B5A',
       '2BF5', '2G5C', '1P4C', '3H2U', '1Y7M', '1MHN', '1GSK', '1VQO',
       '1CY5'], dtype=object)

# Modeling w Processed Data

# Models with Truth Data (has_X)

In [106]:
simple_quant_features = ['isUnstruct', 'Vkbat', '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U'] 
# to include other features need one-hot encoding
print(features)
target = "is_switch"

['isUnstruct', 'Vkbat', '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U']


In [141]:
# Modeling with KNN

# split data
train_data, test_data = train_test_split(df_mo, test_size=0.25, random_state=3)
X_train = train_data[simple_quant_features]
y_train = train_data[target]
X_test = test_data[simple_quant_features]
y_test = test_data[target]

# Classic Logistic Regression Model with all quantitative features

In [114]:
# Fit the model on training data, predict on test data
true_logistic_model = LogisticRegression().fit(X_train, y_train)

In [109]:
preds = true_logistic_model.predict(X_test)
prob_preds = true_logistic_model.predict_proba(X_test)
                               

In [115]:
print(simple_quant_features)
print("Coefficients: " + str(true_logistic_model.coef_))
print("Intercept: " + str(true_logistic_model.intercept_))

['isUnstruct', 'Vkbat', '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U']
Coefficients: [[0.06458752 0.02922285 0.14963802 7.03408216 7.12903188 8.06912582
  7.82577784]]
Intercept: [-11.86497925]


In [116]:
print(preds)
print(y_test)
print((preds - y_test).sum())
print(true_logistic_model.score(X_test, y_test))

[1 0 1 ... 0 0 1]
4491    1
1833    0
381     1
302     0
4206    0
       ..
485     0
3683    0
3675    0
3711    0
4329    1
Name: is_switch, Length: 1259, dtype: int64
0
1.0


# L1 Logistic Model 

In [117]:
l1_true_logistic = LogisticRegression(penalty = 'l1', solver='liblinear').fit(X_train, y_train)
print(simple_quant_features)
print("Coefficients: " + str(l1_true_logistic.coef_))
print("Y int: " + str(l1_true_logistic.intercept_))

['isUnstruct', 'Vkbat', '# homologues', 'HAS_H', 'HAS_S', 'HAS_O', 'HAS_U']
Coefficients: [[ 0.          0.          0.06055831 10.44421504 10.56075658 11.51060529
  11.30390358]]
Y int: [-16.9174108]


In [118]:
l1_preds = l1_true_logistic.predict(X_test)
l1_prob_preds = l1_true_logistic.predict_proba(X_test)
print(l1_preds)
print(y_test)
print((l1_preds - y_test).sum())
print(l1_true_logistic.score(X_test, y_test))
                                    

[1 0 1 ... 0 0 1]
4491    1
1833    0
381     1
302     0
4206    0
       ..
485     0
3683    0
3675    0
3711    0
4329    1
Name: is_switch, Length: 1259, dtype: int64
0
1.0


# Truth Logistic Model 

In [119]:
has_X_features = ['HAS_H', 'HAS_S', 'HAS_O', 'HAS_U']
has_logistic_model = LogisticRegression().fit(train_data[has_X_features], y_train)
print(has_X_features)
print("Coefficients " + str(has_logistic_model.coef_))
print("Intercepts " + str(has_logistic_model.intercept_))
print("Score ")
has_logistic_model.score(test_data[has_X_features], y_test)

['HAS_H', 'HAS_S', 'HAS_O', 'HAS_U']
Coefficients [[7.07280552 7.12430609 8.06458427 7.88434781]]
Intercepts [-11.89568281]
Score 


1.0

# Models without Truth Data

# Classic Logistic Regression

In [130]:
features = ['isUnstruct', 'Vkbat', '# homologues', 'Res_numeric',
       'chou_fasman_Helix', 'chou_fasman_Other', 'chou_fasman_Sheet',
       'sspro_5_Helix', 'sspro_5_Other', 'sspro_5_Sheet', 'gor4_Helix',
       'gor4_Other', 'gor4_Sheet', 'dsc_Helix', 'dsc_Other', 'dsc_Sheet',
       'jnet_Helix', 'jnet_Other', 'jnet_Sheet', 'psipred_Helix',
       'psipred_Other', 'psipred_Sheet']
# columns from df_mo

In [142]:
logistic_model = LogisticRegression().fit(train_data[features], train_data[target])
print(features)
print("Coefficients " + str(logistic_model.coef_))
print("Intercepts " + str(logistic_model.intercept_))


['isUnstruct', 'Vkbat', '# homologues', 'Res_numeric', 'chou_fasman_Helix', 'chou_fasman_Other', 'chou_fasman_Sheet', 'sspro_5_Helix', 'sspro_5_Other', 'sspro_5_Sheet', 'gor4_Helix', 'gor4_Other', 'gor4_Sheet', 'dsc_Helix', 'dsc_Other', 'dsc_Sheet', 'jnet_Helix', 'jnet_Other', 'jnet_Sheet', 'psipred_Helix', 'psipred_Other', 'psipred_Sheet']
Coefficients [[ 0.22855831  0.05449682  0.31762151  0.03650533 -0.01337881 -0.01463372
   0.02745741 -0.59087219  0.38401604  0.20630103  0.13614091 -0.03176364
  -0.10493239 -0.12056549  0.12698645 -0.00697608  0.18578139 -0.12490416
  -0.06143235 -0.06832222  0.01605591  0.05171118]]
Intercepts [-0.80800389]


In [143]:
print("Classic (L2) Score ")
logistic_model.score(test_data[features], test_data[target])

Classic (L2) Score 


0.6719618745035743

# L1 Logistic Regression

In [144]:
l1_logistic_model = LogisticRegression(penalty = 'l1', solver='liblinear').fit(train_data[features], train_data[target])
print(features)
print("Coefficients " + str(l1_logistic_model.coef_))
print("Intercepts " + str(l1_logistic_model.intercept_))

['isUnstruct', 'Vkbat', '# homologues', 'Res_numeric', 'chou_fasman_Helix', 'chou_fasman_Other', 'chou_fasman_Sheet', 'sspro_5_Helix', 'sspro_5_Other', 'sspro_5_Sheet', 'gor4_Helix', 'gor4_Other', 'gor4_Sheet', 'dsc_Helix', 'dsc_Other', 'dsc_Sheet', 'jnet_Helix', 'jnet_Other', 'jnet_Sheet', 'psipred_Helix', 'psipred_Other', 'psipred_Sheet']
Coefficients [[ 0.22580122  0.05463797  0.31604552  0.03486334 -0.03014048 -0.03057812
   0.         -0.96876202  0.         -0.16054288  0.         -0.15208952
  -0.22709656 -0.22789698  0.         -0.11304185  0.13429178 -0.16667595
  -0.09703184 -0.09193342 -0.00654368  0.        ]]
Intercepts [-0.09166376]


In [145]:
print("L1 Score ")
l1_logistic_model.score(test_data[features], test_data[target])

L1 Score 


0.6711675933280381

# Cross Validation

# Other Models (decision tree?)

# Test ROC curves

In [34]:
from sklearn.metrics import roc_curve, auc
from sklearn.datasets import make_classification

In [40]:
print(l1_prob_preds)

[[0.01678671 0.98321329]
 [0.98154929 0.01845071]
 [0.04138236 0.95861764]
 ...
 [0.99298866 0.00701134]
 [0.98507771 0.01492229]
 [0.04334624 0.95665376]]


In [42]:
fpr, tpr, thresholds = roc_curve(y_test, l1_preds)

# need probability scores for this step

fig_hist = px.histogram(
    x=l1_preds, color=y_test, nbins=50,
    labels=dict(color='True Labels', x='Score')
)
fig_hist.show()